In [6]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import keras     # type: ignore
import tensorflow # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from sklearn.preprocessing import LabelEncoder # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from keras.models import Sequential # type: ignore
from keras.layers import Embedding, Flatten, Dense # type: ignore

data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [7]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer =  Tokenizer()
tokenizer.fit_on_texts(texts)

In [8]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [9]:
# Encode the String labels to integer
label_encoder =  LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
# Split the data into training and testing tests
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2)

In [13]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 23s 52ms/step - accuracy: 0.4006 - loss: 1.5180 - val_accuracy: 0.7325 - val_loss: 0.7785
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 24s 61ms/step - accuracy: 0.8797 - loss: 0.3852 - val_accuracy: 0.8138 - val_loss: 0.5707
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 52ms/step - accuracy: 0.9870 - loss: 0.0579 - val_accuracy: 0.8191 - val_loss: 0.6007
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.9957 - loss: 0.0207 - val_accuracy: 0.8188 - val_loss: 0.6530
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.9974 - loss: 0.0169 - val_accuracy: 0.8178 - val_loss: 0.6486
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 55ms/step - accuracy: 0.9979 - loss: 0.0116 - val_accuracy: 0.8156 - val_loss: 0.7246
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 53ms/step - accuracy: 0.9976 - loss: 0.0123 - val_accuracy: 0.8125 - val_loss: 0.7532
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 54ms/step - accuracy: 0.9978 - loss: 0.0095 - 

In [20]:
input_text = "She didn't come today because she lost her dog yesterday!"

# Preprocess the input text
input_sequences = tokenizer.texts_to_sequences([input_text])
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_length)
prediction = model.predict(padded_input_sequences)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
sadness
